In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import scipy.linalg

from lj import *

In [2]:
coord_min = -1
coord_max = 1
np.random.seed(12345)

def run_gradient_based_optimization(N, trials = 100):
    """Returns the minimum energy found over trials."""
    
    min_energy = np.inf
    min_config = None
    
    for t in range(trials):
        initial_points = [np.random.uniform(coord_min, coord_max) for i in range(3*N)]
        res = scipy.optimize.minimize(lennard_jones, initial_points, method='CG', jac=compute_jacobian)
        
        if res.fun < min_energy:
            min_energy = res.fun
            min_config = res.x
            
    return min_config, min_energy
    
def write_configuration(N, min_config, min_energy, is_grad):
    if is_grad:
        with open(f"gradient_{N}.txt", 'w') as f:
            f.write(f"{min_energy}\n")
            f.write(f"{np.reshape(min_config, (-1, 3))}")
    else:
        with open(f"stochastic_{N}.txt", 'w') as f:
            f.write(f"{min_energy}\n")
            f.write(f"{np.reshape(min_config, (-1, 3))}")

In [3]:
N = 8
min_config, min_energy = run_gradient_based_optimization(N)
write_configuration(N, min_config, min_energy, True)

In [4]:
N = 16
min_config, min_energy = run_gradient_based_optimization(N)
write_configuration(N, min_config, min_energy, True)

In [ ]:
N = 32
min_config, min_energy = run_gradient_based_optimization(N)
write_configuration(N, min_config, min_energy, True)

In [23]:
def cmaes(f, x, sigma, g_max, Trace=False):
	def cumulation(c, A, B):
		alpha = 1 - c
		beta = math.sqrt(c * (2 - c) * mueff)
		return [alpha * a + beta * b for a, b in zip(A, B)]

	def wsum(A):
		return [
		    math.fsum(w * a[i] for w, a in zip(weights, A)) for i in range(N)
		]

	xmean, N = x[:], len(x)
	lambd = 4 + int(3 * math.log(N))
	mu = lambd // 2
	weights = [math.log((lambd + 1) / 2) - math.log(i + 1) for i in range(mu)]
	weights = [e / math.fsum(weights) for e in weights]
	mueff = 1 / math.fsum(e**2 for e in weights)
	cc = (4 + mueff / N) / (N + 4 + 2 * mueff / N)
	cs = (mueff + 2) / (N + mueff + 5)
	c1 = 2 / ((N + 1.3)**2 + mueff)
	cmu = min(1 - c1, 2 * (mueff - 2 + 1 / mueff) / ((N + 2)**2 + mueff))
	damps = 1 + 2 * max(0, math.sqrt((mueff - 1) / (N + 1)) - 1) + cs
	chiN = math.sqrt(2) * math.gamma((N + 1) / 2) / math.gamma(N / 2)
	ps, pc, C = [0] * N, [0] * N, np.identity(N)
	trace = []
	for gen in range(1, g_max + 1):
		sqrtC = np.real(scipy.linalg.sqrtm(C))
		x0 = [[random.gauss(0, 1) for d in range(N)] for i in range(lambd)]
		x1 = [sqrtC @ e for e in x0]
		xs = [xmean + sigma * e for e in x1]
		ys = [f(e) for e in xs]
		ys, x0, x1, xs = zip(*sorted(zip(ys, x0, x1, xs)))
		xmean = wsum(xs)
		ps = cumulation(cs, ps, wsum(x0))
		pssq = math.fsum(e**2 for e in ps)
		sigma *= math.exp(cs / damps * (math.sqrt(pssq) / chiN - 1))
		Cmu = sum(w * np.outer(d, d) for w, d in zip(weights, x1))
		if (N + 1) * pssq < 2 * N * (N + 3) * (1 - (1 - cs)**(2 * gen)):
			pc = cumulation(cc, pc, wsum(x1))
			C1 = np.outer(pc, pc)
			C = (1 - c1 - cmu) * C + c1 * C1 + cmu * Cmu
		else:
			pc = [(1 - cc) * e for e in pc]
			C1 = np.outer(pc, pc)
			C = (1 - c1 - cmu) * C + c1 * (C1 + cc * (2 - cc) * C) + cmu * Cmu
		if Trace:
			trace.append(
			    (gen * lambd, ys[0], xs[0], sigma, C, ps, pc, Cmu, C1, xmean))
	return trace if Trace else xmean

In [67]:
def run_cmaes_optimization(N, sigma, ngen=100, trials = 100, coord_min = -1, coord_max = 1):
    """Returns the minimum energy found over trials."""
    
    min_energy = np.inf
    min_config = None
    
    for t in range(trials):
        # if t % 10 == 0 or t == trials - 1:
        print(t)
        initial_points = [np.random.uniform(coord_min, coord_max) for i in range(3*N)]
        xs = cmaes(lennard_jones, initial_points, sigma, ngen)
        val = lennard_jones(xs)
        
        if val < min_energy:
            min_energy = val
            min_config = xs
            
    return min_config, min_energy

In [59]:
N = 8
sigma = 0.1
ngen = 500
min_config, min_energy = run_cmaes_optimization(N, sigma, ngen, trials = 50)
print(min_energy)
write_configuration(N, min_config, min_energy, False)

0
10
20
30
40
49
-19.821489192154758


In [60]:
N = 16
sigma = 0.05
ngen = 500
min_config, min_energy = run_cmaes_optimization(N, sigma, ngen, trials = 50)
print(min_energy)
write_configuration(N, min_config, min_energy, False)

0
10
20
30
40
49
-55.84871994671202


In [68]:
N = 32
sigma = 0.1
ngen = 500
min_config, min_energy = run_cmaes_optimization(N, sigma, ngen, trials = 10, coord_min=-1, coord_max=1)
print(min_energy)
write_configuration(N, min_config, min_energy, False)

0
1
2
3
4
5
6
7
8
9
-122.13671470633275


In [70]:
N = 38
min_config, min_energy = run_gradient_based_optimization(N)
write_configuration(N, min_config, min_energy, True)

In [69]:
N = 38
sigma = 0.1
ngen = 500
min_config, min_energy = run_cmaes_optimization(N, sigma, ngen, trials = 1, coord_min=-1.5, coord_max=1.5)
print(min_energy)
write_configuration(N, min_config, min_energy, False)

0
-142.26336234039837
